# Geocoding apartment addresses using the geoadmin API

## Libraries and settings

In [1]:
# Libraries
import os
import requests
import json
import urllib
import fnmatch
import folium
import pandas as pd
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Get current working directory
print(os.getcwd())

c:\Users\Jasmin\Desktop\DA\data_analytics\Week_05


## Geocoding a single address

### Define base url for address search

In [2]:
# Define base url for address search
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Set up search parameters: address, origins and type
parameters = {"searchText": "Stationsstrasse 2, Zell",
              "origins": "address",
              "type": "locations",
             }

# Urllib.parse.urlencode turns parameters into url
url = f"{base_url}{urllib.parse.urlencode(parameters)}"
print(url)

https://api3.geo.admin.ch/rest/services/api/SearchServer?searchText=Stationsstrasse+2%2C+Zell&origins=address&type=locations


### Server request & response

In [3]:
# Server request
r = requests.get(url)

# Get data in json-format
data = json.loads(r.content)
data

# Take only the first server response, convert to data frame with relevant infos
df = pd.DataFrame.from_dict(list(data.values())[0][0], orient='columns')
df.iloc[[1,4,5,6,11,12],:1]

,attrs
featureId,115075_0
label,Stationsstrasse 2 <b>8487 Zell ZH</b>
lat,47.447277
lon,8.822617
x,256085.765625
y,704377.75


## Geocoding multiple addresses

### Importing apartment data

In [4]:
# Get current working directory
print(os.getcwd())

# Show all files in the directory
flist = fnmatch.filter(os.listdir('.'), '*.csv')
for i in flist:
    print(i)

# Read the data to a pandas data frame
df = pd.read_csv('apartments_data_prepared.csv', 
                 sep=',', 
                 encoding='utf-8')[['web-scraper-order', 
                                    'address_raw', 
                                    'datetime',
                                    'rooms', 
                                    'area', 
                                    'luxurious',
                                    'price_per_m2']][:100] # first 100 apartment adresses

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(10)

c:\Users\Jasmin\Desktop\DA\data_analytics\Week_05
apartments_data_prepared.csv
supermarkets_data_prepared.csv
(100, 7)


,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price_per_m2
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",2023-09-06 15:10:34,3.0,49.0,0,29.41
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",2023-09-06 15:10:34,3.5,65.0,0,28.46
2,1693998201-4,"Cramerstrasse 8-12, 8004 Zürich, ZH",2023-09-06 15:10:34,2.0,54.0,0,89.87
3,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",2023-09-06 15:10:34,2.0,49.0,0,88.47
4,1693998201-16,"Wolframplatz 1, 8045 Zürich, ZH",2023-09-06 15:10:34,2.0,32.0,0,109.84
5,1693998205-25,"Badenerstrasse 67, 8953 Dietikon, ZH",2023-09-06 15:10:34,3.5,105.0,0,30.48
6,1693998205-26,"Frowiesstrasse 36, 8344 Bäretswil, ZH",2023-09-06 15:10:34,5.5,140.0,0,19.64
7,1693998205-27,"Kanzleistrasse 15, 8418 Schlatt ZH, ZH",2023-09-06 15:10:34,4.5,104.0,0,25.96
8,1693998205-29,"Bolletweg 14, 8934 Knonau, ZH",2023-09-06 15:10:34,5.5,150.0,0,22.60
9,1693998205-30,"Im Zauner 24, 8352 Elsau, ZH",2023-09-06 15:10:34,3.0,70.0,0,35.00


### Geocoding multiple apartment addresses using the geoadmin API

In [14]:
# Define base url
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Geocode list of adresses
geolocation = []
n = 1
for i in df['address_raw'].astype(str):
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df['address_raw']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
        # Set up search parameters - address, origins and type
        parameters = {"searchText": i,
                      "origins": "address",
                      "type": "locations",
                     }

        # Server request
        r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

        # Get data
        data = json.loads(r.content)

        # Take first server response, convert to df with relevant infos
        df_loc = pd.DataFrame.from_dict(list(data.values())[0][0], 
                                        orient='columns')
        geolocation.append(df_loc.iloc[[5,6],0].astype(float))
    
    except:
        geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
        

# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df['address_raw'])))
df['lat'] = df_loc['lat']
df['lon'] = df_loc['lon']
df.head(5)

,web-scraper-order,address_raw,datetime,rooms,area,luxurious,price_per_m2,lat,lon
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",2023-09-06 15:10:34,3.0,49.0,0,29.41,47.252167,8.845798
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",2023-09-06 15:10:34,3.5,65.0,0,28.46,47.513332,8.474851
2,1693998201-4,"Cramerstrasse 8-12, 8004 Zürich, ZH",2023-09-06 15:10:34,2.0,54.0,0,89.87,NaN,NaN
3,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",2023-09-06 15:10:34,2.0,49.0,0,88.47,47.370792,8.514748
4,1693998201-16,"Wolframplatz 1, 8045 Zürich, ZH",2023-09-06 15:10:34,2.0,32.0,0,109.84,47.362282,8.522193


### Plot addresses on map

In [15]:
# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Add lat/lon of addresses
df_sub = df.dropna()
for i in range(0, len(df_sub)):
    folium.Marker(location=(df_sub.iloc[i]['lat'], 
                            df_sub.iloc[i]['lon']), 
                  popup=df_sub.iloc[i]['address_raw']).add_to(m)

# Layer control
folium.LayerControl().add_to(m)

# Plot map
m

### Save data to file

In [16]:
df.to_csv('Geodata/apartments_data_geocoded.csv', 
           sep=",", 
           encoding='utf-8',
           index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [8]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2024-11-14 14:53:55
Python Version: 3.12.0
-----------------------------------
